# HA4

In [64]:
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
from lxml import html
from datetime import date

In [65]:
# запускаем базу news и открываем там коллекцию yandex_news
from pymongo import MongoClient
client = MongoClient('127.0.0.1', 27017)

db = client['news']
yandex_news = db.yandex_news

In [66]:
# скрапим данные с яндекса
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36',
          'Accept' : '*/*'}

url = 'https://yandex.ru/news'
response = requests.get(url, headers = headers)
dom = html.fromstring(response.text)

In [67]:
# парсим заголовки статей
data = dom.xpath("//div[contains(@class, 'mg-grid__row mg-grid__row_gap_8 news-top-flexible-stories')]/div[contains(@class, 'mg-grid__col mg-grid__col_xs_')]//h2[contains(@class, 'mg-card')]/text()")

title = []
for item in data:
    title.append(item.replace("\xa0", " "))

# парсим ссылки на статьи на самом агрегаторе
link = dom.xpath("//div[contains(@class, 'mg-grid__row mg-grid__row_gap_8 news-top-flexible-stories')]/div[contains(@class, 'mg-grid__col mg-grid__col_xs_')]//a[contains(@class, 'mg-card__link')]/@href")

# парсим источники
source = dom.xpath("//div[contains(@class, 'mg-grid__row mg-grid__row_gap_8 news-top-flexible-stories')]/div[contains(@class, 'mg-grid__col mg-grid__col_xs_')]//a[contains(@class, 'mg-card__source-link')]/text()")

# парсим дату новости
data = dom.xpath("//div[contains(@class, 'mg-grid__row mg-grid__row_gap_8 news-top-flexible-stories')]/div[contains(@class, 'mg-grid__col mg-grid__col_xs_')]//span[contains(@class, 'mg-card-source__time')]/text()")

issue_date = []
for i in data:
    time = datetime.strptime(i, '%H:%M').time()
    issue_date.append(datetime.combine(date.today(), time))


In [68]:
df = pd.DataFrame(list(zip(title, link, source, issue_date)), 
                  columns = ['Заголовок', 'Ссылка', 'Источник', 'Дата'])

df.to_csv('news.csv', index=False, sep= ',',encoding='utf-8-sig')
df.head()

,Заголовок,Ссылка,Источник,Дата
0,Два самолета МО РФ доставили эвакуированных из...,https://yandex.ru/news/story/Dva_samoleta_MO_R...,Известия,2021-08-25 17:55:00
1,Путин разрешил допустить к госслужбе россиян с...,https://yandex.ru/news/story/Putin_razreshil_d...,Ведомости,2021-08-25 17:52:00
2,Подозреваемого в убийстве восьмилетней девочки...,https://yandex.ru/news/story/Podozrevaemogo_vu...,ТАСС,2021-08-25 17:49:00
3,Nord Stream 2 проиграла дело о выводе «Северно...,https://yandex.ru/news/story/Nord_Stream_2_pro...,РБК,2021-08-25 17:50:00
4,Сопротивление талибам в Афганистане призвало Р...,https://yandex.ru/news/story/Soprotivlenie_tal...,Известия,2021-08-25 17:48:00


In [69]:
double = 0
for i in range(len(df)):    
    if yandex_news.count_documents({'link' : df['Ссылка'].iloc[i]}) == 0: 
        yandex_news.insert_one({'title' : df['Заголовок'].iloc[i],
            'link' : df['Ссылка'].iloc[i],                          
            'source' : df['Источник'].iloc[i],
            'datetime' : df['Дата'].iloc[i]})
    else:
            double += 1

print(f' Всего новостей в базе: {yandex_news.estimated_document_count()}, \n было добавлено: {len(df)-double} новостей, \n было выявлено дублей: {double}, они не попали в базу')

 Всего новостей в базе: 10, 
 было добавлено: 5 новостей, 
 было выявлено дублей: 0, они не попали в базу


In [70]:
# выводим содержимое базы
for i in yandex_news.find({}):
    pprint(i)

{'_id': ObjectId('612659f70af98dfe2b42f393'),
 'datetime': datetime.datetime(2021, 8, 25, 17, 40),
 'link': 'https://yandex.ru/news/story/Dva_samoleta_MO_RF_dostavili_ehvakuirovannykh_izAfganistana_grazhdan_Kirgizii--f38b49e7e01acbcd79a1be0fba39ef19?lang=ru&rubric=index&fan=1&stid=R3c3xEdCTWc1uCgaluYQ&t=1629902672&tt=true&persistent_id=158460989',
 'source': 'Известия',
 'title': 'Два самолета МО РФ доставили эвакуированных из Афганистана граждан '
          'Киргизии'}
{'_id': ObjectId('612659f70af98dfe2b42f394'),
 'datetime': datetime.datetime(2021, 8, 25, 17, 38),
 'link': 'https://yandex.ru/news/story/Podozrevaemogo_vubijstve_vosmiletnej_devochki_vTyumeni_zaderzhali--6a8a5a25aba7949a4cc31026d61602f2?lang=ru&rubric=index&fan=1&stid=0Lz9QyEkSzo36VZluwdC&t=1629902672&tt=true&persistent_id=159211683',
 'source': 'ТАСС',
 'title': 'Подозреваемого в убийстве восьмилетней девочки в Тюмени задержали'}
{'_id': ObjectId('612659f70af98dfe2b42f395'),
 'datetime': datetime.datetime(2021, 8, 25,

In [71]:
#yandex_news.delete_many({})
#yandex_news.estimated_document_count()
